# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [2]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [4]:
from crewai import Agent, Task, Crew
import os
# Set your OpenAI API key directly
os.environ["OPENAI_API_KEY"] = "Your OPENAI API KEY"
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = " Your SERPER API KEY"

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [ ]:
# import os
# from utils import get_openai_api_key, get_serper_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
# os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [8]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [10]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [12]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [14]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [16]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [18]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [20]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [22]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [24]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [32]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'TESLA',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [34]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (TESLA). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for TESLA to identify trends and predict market movements. I should delegate tasks based on each team member's expertise and ask questions to gather all necessary information.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for TESLA", "context": "Utilize statistical modeling and machine learning to identify trends and predict market movements"}

> Entering new CrewAgentExecutor chain...
Thought: To monitor and analyze market data for TESLA, I need real-time or the most recent data related to its stock performance, news, and any relevant market tre

- Display the final result as Markdown.

In [36]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report:

Introduction:
The following risk analysis report aims to assess the potential risks associated with the proposed trading strategies and execution plans for Tesla (TSLA). By evaluating key factors such as market volatility, sentiment, legal and regulatory challenges, and global economic influences, this report provides recommendations for effective risk mitigation strategies.

Risk Assessment:

1. Volatility:
Tesla's stock is characterized by high volatility, which can lead to significant price fluctuations. This poses a risk to traders, particularly those engaged in short-term trading strategies. To mitigate this risk, traders should consider implementing tighter stop-loss orders and closely monitoring the market for any abnormal activity that may impact stock prices.

2. Market Sentiment:
Tesla's stock is highly influenced by market sentiment, which can be affected by news related to the company, technological advancements, and public statements made by key figures such as Elon Musk. It is crucial for traders to stay informed by regularly monitoring news updates and analyst reports to anticipate and respond to potential market movements.

3. Legal and Regulatory Issues:
Tesla has encountered various legal challenges and regulatory scrutiny, which can have a direct impact on stock prices. Traders must remain vigilant of any ongoing legal issues or regulatory changes that could influence Tesla's business operations or market perception.

4. Global Economic Factors:
Changes in tariffs, international trade relations, and disruptions in the supply chain can also pose risks to Tesla's stock performance. Traders should stay informed about global economic news and its potential impact on the automotive and technology sectors to effectively manage these risks.

Mitigation Strategies:

To effectively manage these risks, the following mitigation strategies are recommended:

1. Utilize technical analysis to identify stable entry and exit points, reducing exposure during periods of high volatility.
2. Implement robust risk management strategies, including diversifying the portfolio allocation and regularly reviewing exposure levels to align with the firm's risk tolerance.
3. Stay abreast of Tesla-specific news and broader economic indicators to make informed trading decisions that are aligned with the current market conditions.

Conclusion:

By proactively addressing these risks with the recommended mitigation strategies, traders can enhance their ability to capitalize on trading opportunities while safeguarding their investments against unforeseen downturns in Tesla's stock. It is essential for traders to remain vigilant, adapt to changing market conditions, and continuously reassess their risk management strategies to achieve optimal trading outcomes.